# Regression Analysis 

### Author_: Christian Rosado

In [419]:
# imports
from __future__  import print_function, division
import pylab as pl
import pandas as pd
import numpy as np
#from geopandas import GeoDataFrame
import geopandas as gpd
import matplotlib.pyplot as plt
from fiona.crs import from_epsg
#import shapely as shp
import seaborn 
import os
import urllib
import requests
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
#Pulling Data via API
wifi_data = pd.read_json("http://wifindproject.com/wifipulling/?device_model=VS425")

In [3]:
wifi_data.head()

,acc,altitude,app_version,bssid,caps,device_mac,device_model,droid_version,freq,idx,lat,level,lng,ssid,time
0,19.957,0.0,0.3.3,1c:e8:5d:ca:8f:33,[wpa-eap-tkip][wpa2-eap-ccmp][ess],d3a2898a5e731607,VS425,5.1.1,2437,56629534,40.690966,-60,-73.987686,nyu-legacy,1497296993826
1,19.957,0.0,0.3.3,1c:e8:5d:ca:8f:32,[wpa2-eap-ccmp][ess],d3a2898a5e731607,VS425,5.1.1,2437,56629535,40.690966,-59,-73.987686,eduroam,1497296993826
2,19.957,0.0,0.3.3,cc:d5:39:27:f2:43,[wpa-eap-tkip][wpa2-eap-ccmp][ess],d3a2898a5e731607,VS425,5.1.1,2412,56629536,40.690966,-66,-73.987686,nyu-legacy,1497296993826
3,19.957,0.0,0.3.3,cc:d5:39:27:f2:42,[wpa2-eap-ccmp][ess],d3a2898a5e731607,VS425,5.1.1,2412,56629537,40.690966,-67,-73.987686,eduroam,1497296993826
4,19.957,0.0,0.3.3,00:a2:ee:03:e3:20,[wpa2-eap-ccmp][ess],d3a2898a5e731607,VS425,5.1.1,2412,56629538,40.690966,-73,-73.987686,constitution,1497296993826


In [4]:
# filter records with accuaracy scores of 50 and above
wifi_data = wifi_data.drop(wifi_data[wifi_data.acc > 50].index)

In [5]:
wifi_data.head()

,acc,altitude,app_version,bssid,caps,device_mac,device_model,droid_version,freq,idx,lat,level,lng,ssid,time
0,19.957,0.0,0.3.3,1c:e8:5d:ca:8f:33,[wpa-eap-tkip][wpa2-eap-ccmp][ess],d3a2898a5e731607,VS425,5.1.1,2437,56629534,40.690966,-60,-73.987686,nyu-legacy,1497296993826
1,19.957,0.0,0.3.3,1c:e8:5d:ca:8f:32,[wpa2-eap-ccmp][ess],d3a2898a5e731607,VS425,5.1.1,2437,56629535,40.690966,-59,-73.987686,eduroam,1497296993826
2,19.957,0.0,0.3.3,cc:d5:39:27:f2:43,[wpa-eap-tkip][wpa2-eap-ccmp][ess],d3a2898a5e731607,VS425,5.1.1,2412,56629536,40.690966,-66,-73.987686,nyu-legacy,1497296993826
3,19.957,0.0,0.3.3,cc:d5:39:27:f2:42,[wpa2-eap-ccmp][ess],d3a2898a5e731607,VS425,5.1.1,2412,56629537,40.690966,-67,-73.987686,eduroam,1497296993826
4,19.957,0.0,0.3.3,00:a2:ee:03:e3:20,[wpa2-eap-ccmp][ess],d3a2898a5e731607,VS425,5.1.1,2412,56629538,40.690966,-73,-73.987686,constitution,1497296993826


# Needed variables:
* Total Population
* Population Density
* House Hold Density
* Median Household Income
* Wifi Counts (open and other)
* Location (lat, long)
    

In [450]:
# 2010 & 2000 pop by census tract - BK
#NYC_pop_data = pd.read_json("https://data.cityofnewyork.us/resource/rx2e-87id.json", dtype={'census_tract': str})
NYC_pop_data = pd.read_csv("https://data.cityofnewyork.us/api/views/rx2e-87id/rows.csv?accessType=DOWNLOAD", dtype={'Census Tract': str})

In [451]:
NYC_pop_data.shape

(1522, 6)

In [452]:
NYC_pop_data.head()

,Borough,Year,FIPS County Code,DCP Borough Code,Census Tract,Population
0,Brooklyn,2000,47,3,000100,5498
1,Brooklyn,2000,47,3,000200,1303
2,Brooklyn,2000,47,3,000301,5036
3,Brooklyn,2000,47,3,000501,3415
4,Brooklyn,2000,47,3,000502,2739


In [453]:
NYC_pop_data.drop(NYC_pop_data[NYC_pop_data.Year == 2000].index, inplace=True)

In [454]:
NYC_pop_data.head()

,Borough,Year,FIPS County Code,DCP Borough Code,Census Tract,Population
761,Brooklyn,2010,47,3,000100,4338
762,Brooklyn,2010,47,3,000200,1470
763,Brooklyn,2010,47,3,000301,4259
764,Brooklyn,2010,47,3,000501,4247
765,Brooklyn,2010,47,3,000502,2512


In [455]:
NYC_pop_data.shape

(761, 6)

In [456]:
NYC_pop_data["boro_ct2010"] = NYC_pop_data["DCP Borough Code"].astype(str) + NYC_pop_data["Census Tract"].astype(str)

In [457]:
NYC_pop_data["boro_ct2010"] = NYC_pop_data["boro_ct2010"].astype(int)

In [458]:
NYC_pop_data["boro_ct2010"].dtype

dtype('int64')

In [459]:
NYC_pop_data.Borough.unique()

array(['Brooklyn'], dtype=object)

In [460]:
# 6 digits
# leading with 2 zeros for 2 digit numbers
# leading with 1 zeros for 3 digit numbers
# ending with 2 zeros for decimal places

NYC_pop_data[NYC_pop_data["Census Tract"].str.contains("0")].head()

,Borough,Year,FIPS County Code,DCP Borough Code,Census Tract,Population,boro_ct2010
761,Brooklyn,2010,47,3,000100,4338,3000100
762,Brooklyn,2010,47,3,000200,1470,3000200
763,Brooklyn,2010,47,3,000301,4259,3000301
764,Brooklyn,2010,47,3,000501,4247,3000501
765,Brooklyn,2010,47,3,000502,2512,3000502


In [461]:
NYC_pop_data.shape

(761, 7)

In [546]:
# 2010 Census Tracts - NYC Open Data
# https://data.cityofnewyork.us/City-Government/2010-Census-Tracts/fxpq-c8ku

"""REFERENCE SYSTEM IDENTIFIER
* VALUE 2263
* CODESPACE EPSG
* VERSION 8.6.2"""

#data = requests.get("https://data.cityofnewyork.us/resource/i69b-3rdj.json")
data = "https://data.cityofnewyork.us/api/views/i69b-3rdj/rows.csv?accessType=DOWNLOAD"
df = pd.read_csv(data)
gdf = gpd.GeoDataFrame(df) #(data.json())
gdf.head()

,BoroName,BoroCode,the_geom,CTLabel,CT2010,BoroCT2010,CDEligibil,NTACode,NTAName,PUMA,Shape_Leng,Shape_Area
0,Staten Island,5,MULTIPOLYGON (((-74.07920577013245 40.64343078...,9.0,900,5000900,I,SI22,West New Brighton-New Brighton-St. George,3903,7729.016794,2.497010e+06
1,Manhattan,1,MULTIPOLYGON (((-73.96432543478758 40.75638153...,98.0,9800,1009800,I,MN19,Turtle Bay-East Midtown,3808,5534.199811,1.906016e+06
2,Manhattan,1,MULTIPOLYGON (((-73.96802436915851 40.75957814...,100.0,10000,1010000,I,MN19,Turtle Bay-East Midtown,3808,5692.168737,1.860938e+06
3,Manhattan,1,MULTIPOLYGON (((-73.97124277307127 40.76093641...,102.0,10200,1010200,I,MN17,Midtown-Midtown South,3807,5687.802439,1.860993e+06
4,Manhattan,1,MULTIPOLYGON (((-73.97445730550224 40.76229308...,104.0,10400,1010400,I,MN17,Midtown-Midtown South,3807,5693.036367,1.864600e+06


In [552]:
gdf["boro_ct2010"] = gdf["BoroCT2010"].astype(int)

In [555]:
gdf[gdf.BoroCode == 3].shape

(760, 13)

In [556]:
#country_shapes = country_shapes.merge(country_names, on='iso_a3')
NYC_shape = gdf.merge(NYC_pop_data, on="boro_ct2010")

In [557]:
NYC_shape.shape

(760, 19)

In [558]:
NYC_shape.head()

,BoroName,BoroCode,the_geom,CTLabel,CT2010,BoroCT2010,CDEligibil,NTACode,NTAName,PUMA,Shape_Leng,Shape_Area,boro_ct2010,Borough,Year,FIPS County Code,DCP Borough Code,Census Tract,Population
0,Brooklyn,3,MULTIPOLYGON (((-73.99301822153384 40.69746236...,5.01,501,3000501,I,BK09,Brooklyn Heights-Cobble Hill,4004,4828.169912,9.015070e+05,3000501,Brooklyn,2010,47,3,000501,4247
1,Brooklyn,3,MULTIPOLYGON (((-74.01266264860631 40.64925555...,20.00,2000,3002000,I,BK32,Sunset Park West,4012,5426.067279,1.620770e+06,3002000,Brooklyn,2010,47,3,002000,1672
2,Brooklyn,3,MULTIPOLYGON (((-73.98181677448231 40.68732980...,41.00,4100,3004100,I,BK38,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,4004,5283.856660,1.744094e+06,3004100,Brooklyn,2010,47,3,004100,3598
3,Brooklyn,3,MULTIPOLYGON (((-74.03809447949381 40.62058489...,52.02,5202,3005202,I,BK31,Bay Ridge,4013,5190.713818,1.398134e+06,3005202,Brooklyn,2010,47,3,005202,2481
4,Brooklyn,3,MULTIPOLYGON (((-74.02956408409689 40.61983253...,58.00,5800,3005800,E,BK31,Bay Ridge,4013,6098.167799,2.080303e+06,3005800,Brooklyn,2010,47,3,005800,3152


In [559]:
NYC_shape.Population.dtype

dtype('int64')

In [561]:
NYC_shape.Shape_Area.dtype

dtype('float64')

In [566]:
NYC_shape.head()

,BoroName,BoroCode,the_geom,CTLabel,CT2010,BoroCT2010,CDEligibil,NTACode,NTAName,PUMA,Shape_Leng,Shape_Area,boro_ct2010,Borough,Year,FIPS County Code,DCP Borough Code,Census Tract,Population
0,Brooklyn,3,MULTIPOLYGON (((-73.99301822153384 40.69746236...,5.01,501,3000501,I,BK09,Brooklyn Heights-Cobble Hill,4004,4828.169912,9.015070e+05,3000501,Brooklyn,2010,47,3,000501,4247
1,Brooklyn,3,MULTIPOLYGON (((-74.01266264860631 40.64925555...,20.00,2000,3002000,I,BK32,Sunset Park West,4012,5426.067279,1.620770e+06,3002000,Brooklyn,2010,47,3,002000,1672
2,Brooklyn,3,MULTIPOLYGON (((-73.98181677448231 40.68732980...,41.00,4100,3004100,I,BK38,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,4004,5283.856660,1.744094e+06,3004100,Brooklyn,2010,47,3,004100,3598
3,Brooklyn,3,MULTIPOLYGON (((-74.03809447949381 40.62058489...,52.02,5202,3005202,I,BK31,Bay Ridge,4013,5190.713818,1.398134e+06,3005202,Brooklyn,2010,47,3,005202,2481
4,Brooklyn,3,MULTIPOLYGON (((-74.02956408409689 40.61983253...,58.00,5800,3005800,E,BK31,Bay Ridge,4013,6098.167799,2.080303e+06,3005800,Brooklyn,2010,47,3,005800,3152


In [567]:
#from fiona.crs import from_epsg
#ct_shape.crs = from_epsg(4326) # epsg=4326: lat/on | 26918: NAD83/UTM zone 18N | epsg=2263 is US feet
#NYC_Area=ct_shape.to_crs(epsg=2263).geometry.area.sum()/(2.788*10**7) # from ft2 to mi2 
#print 'total NYC land area: %s (mi^2)'%str(NYC_Area)[:5] # from ft2 to mi2 #epsg=2263 is US feet

In [568]:
NYC_shape.crs = from_epsg(4326)

In [569]:
NYC_shape.crs

{'init': 'epsg:4326', 'no_defs': True}

In [571]:
# Population density by 1,000 sq feet
NYC_shape["pop_density"] = (NYC_shape.Population/NYC_shape.Shape_Area)*1000

In [572]:
NYC_shape.head()

,BoroName,BoroCode,the_geom,CTLabel,CT2010,BoroCT2010,CDEligibil,NTACode,NTAName,PUMA,Shape_Leng,Shape_Area,boro_ct2010,Borough,Year,FIPS County Code,DCP Borough Code,Census Tract,Population,pop_density
0,Brooklyn,3,MULTIPOLYGON (((-73.99301822153384 40.69746236...,5.01,501,3000501,I,BK09,Brooklyn Heights-Cobble Hill,4004,4828.169912,9.015070e+05,3000501,Brooklyn,2010,47,3,000501,4247,4.711001
1,Brooklyn,3,MULTIPOLYGON (((-74.01266264860631 40.64925555...,20.00,2000,3002000,I,BK32,Sunset Park West,4012,5426.067279,1.620770e+06,3002000,Brooklyn,2010,47,3,002000,1672,1.031608
2,Brooklyn,3,MULTIPOLYGON (((-73.98181677448231 40.68732980...,41.00,4100,3004100,I,BK38,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,4004,5283.856660,1.744094e+06,3004100,Brooklyn,2010,47,3,004100,3598,2.062962
3,Brooklyn,3,MULTIPOLYGON (((-74.03809447949381 40.62058489...,52.02,5202,3005202,I,BK31,Bay Ridge,4013,5190.713818,1.398134e+06,3005202,Brooklyn,2010,47,3,005202,2481,1.774507
4,Brooklyn,3,MULTIPOLYGON (((-74.02956408409689 40.61983253...,58.00,5800,3005800,E,BK31,Bay Ridge,4013,6098.167799,2.080303e+06,3005800,Brooklyn,2010,47,3,005800,3152,1.515164


In [573]:
#building_data = pd.read_json("https://data.cityofnewyork.us/resource/q3m4-ttp3.json", dtype={'census_tract': str}) 

In [574]:
# Shapefile: Median Household Income, Census Tract
#http://data.beta.nyc/dataset/median-household-income-2010-census-tracts/resource/92bbc70e-2b4e-40aa-8a1f-d39b11124b49

path = "../Downloads/medianhouseholdincomecensustract.csv"
house_data = pd.read_csv(path, dtype={'TRACTCE10': str})
house_data.head()

,FID,STATEFP10,COUNTYFP10,TRACTCE10,GEOID10,INTPTLAT10,INTPTLON10,State,COUNTY,HH_COUNT,...,MSMOC_TOT_,TAX_RET,Avg_TransC,REtaxperow,DISP_INC,energy_cos,REtax_ACS,AVG_TTL,LOCALNAME,Shape_Area
0,0,34,17,32400,34017032400,40.792844,-74.013482,New Jersey,Hudson County,2187,...,22740,2264.889916,4145.566539,4966.370331,12232.00950,1773.280152,7735,5007.444405,West New York Town,0.000036
1,1,34,17,10100,34017010100,40.691559,-74.110913,New Jersey,Hudson County,2255,...,27804,2363.430717,5969.286255,4490.984109,22077.84024,1876.844806,7917,6146.888195,Bayonne City,0.000106
2,2,34,17,10200,34017010200,40.682103,-74.104573,New Jersey,Hudson County,1218,...,28068,1260.718928,6014.489564,3339.224580,20488.60531,2167.175106,7692,6193.436421,Bayonne City,0.000031
3,3,34,17,10300,34017010300,40.672439,-74.081016,New Jersey,Hudson County,1093,...,22620,1229.562591,4798.787758,3793.386763,15514.61337,2163.964082,7831,4941.564294,Bayonne City,0.000108
4,4,34,17,10400,34017010400,40.670599,-74.089940,New Jersey,Hudson County,1571,...,20724,2209.874484,7611.970058,4478.110461,35131.00534,2114.480847,7918,7838.446154,Bayonne City,0.000234


In [575]:
house_data.columns

Index(['FID', 'STATEFP10', 'COUNTYFP10', 'TRACTCE10', 'GEOID10', 'INTPTLAT10',
       'INTPTLON10', 'State', 'COUNTY', 'HH_COUNT', 'OHU_OWN', 'OHU_RENT',
       'PERC_OWN', 'PERC_RENT', 'MHI', 'MED_VAL', 'HOUSINGCOS', 'MCR_ANNUAL',
       'MSMOC_TOT_', 'TAX_RET', 'Avg_TransC', 'REtaxperow', 'DISP_INC',
       'energy_cos', 'REtax_ACS', 'AVG_TTL', 'LOCALNAME', 'Shape_Area'],
      dtype='object')

In [576]:
#house_data[house_data.TRACTCE10.str.contains("11")]

In [577]:
#house_data["boro_ct2010"] = house_data.TRACTCE10.str.ljust(5,'0')

In [578]:
house_data["boro_ct2010"] = house_data.TRACTCE10.str.zfill(6)

In [579]:
house_data.head()

,FID,STATEFP10,COUNTYFP10,TRACTCE10,GEOID10,INTPTLAT10,INTPTLON10,State,COUNTY,HH_COUNT,...,TAX_RET,Avg_TransC,REtaxperow,DISP_INC,energy_cos,REtax_ACS,AVG_TTL,LOCALNAME,Shape_Area,boro_ct2010
0,0,34,17,32400,34017032400,40.792844,-74.013482,New Jersey,Hudson County,2187,...,2264.889916,4145.566539,4966.370331,12232.00950,1773.280152,7735,5007.444405,West New York Town,0.000036,032400
1,1,34,17,10100,34017010100,40.691559,-74.110913,New Jersey,Hudson County,2255,...,2363.430717,5969.286255,4490.984109,22077.84024,1876.844806,7917,6146.888195,Bayonne City,0.000106,010100
2,2,34,17,10200,34017010200,40.682103,-74.104573,New Jersey,Hudson County,1218,...,1260.718928,6014.489564,3339.224580,20488.60531,2167.175106,7692,6193.436421,Bayonne City,0.000031,010200
3,3,34,17,10300,34017010300,40.672439,-74.081016,New Jersey,Hudson County,1093,...,1229.562591,4798.787758,3793.386763,15514.61337,2163.964082,7831,4941.564294,Bayonne City,0.000108,010300
4,4,34,17,10400,34017010400,40.670599,-74.089940,New Jersey,Hudson County,1571,...,2209.874484,7611.970058,4478.110461,35131.00534,2114.480847,7918,7838.446154,Bayonne City,0.000234,010400


Manhattan - New York County. 1
Bronx - Bronx County. 2
Brooklyn - Kings County. 3
Queens - Queens County. 4
Staten Island - Richmond County. 5

In [580]:
def boro(x):
    if x == "New York County":
        return 1
    if x == "Bronx County":
        return 2
    if x == "Kings County":
        return 3
    if x == "Queens County":
        return 4
    if x == "Richmond County":
        return 5

In [581]:
boro('New York County')

1

In [582]:
house_data.head()

,FID,STATEFP10,COUNTYFP10,TRACTCE10,GEOID10,INTPTLAT10,INTPTLON10,State,COUNTY,HH_COUNT,...,TAX_RET,Avg_TransC,REtaxperow,DISP_INC,energy_cos,REtax_ACS,AVG_TTL,LOCALNAME,Shape_Area,boro_ct2010
0,0,34,17,32400,34017032400,40.792844,-74.013482,New Jersey,Hudson County,2187,...,2264.889916,4145.566539,4966.370331,12232.00950,1773.280152,7735,5007.444405,West New York Town,0.000036,032400
1,1,34,17,10100,34017010100,40.691559,-74.110913,New Jersey,Hudson County,2255,...,2363.430717,5969.286255,4490.984109,22077.84024,1876.844806,7917,6146.888195,Bayonne City,0.000106,010100
2,2,34,17,10200,34017010200,40.682103,-74.104573,New Jersey,Hudson County,1218,...,1260.718928,6014.489564,3339.224580,20488.60531,2167.175106,7692,6193.436421,Bayonne City,0.000031,010200
3,3,34,17,10300,34017010300,40.672439,-74.081016,New Jersey,Hudson County,1093,...,1229.562591,4798.787758,3793.386763,15514.61337,2163.964082,7831,4941.564294,Bayonne City,0.000108,010300
4,4,34,17,10400,34017010400,40.670599,-74.089940,New Jersey,Hudson County,1571,...,2209.874484,7611.970058,4478.110461,35131.00534,2114.480847,7918,7838.446154,Bayonne City,0.000234,010400


In [583]:
house_data["boro_code"] = house_data.COUNTY.map({"New York County": "1", "Bronx County": "2", "Kings County": "3", \
                                           "Queens County": "4", "Richmond County": "5"})

In [584]:
house_data.boro_ct2010 = house_data.boro_code.astype(str) + house_data.boro_ct2010.astype(str)

In [585]:
#house_data[house_data.boro_code == "3"]

In [586]:
NYC_shape[NYC_shape.boro_ct2010 == 3074000]

,BoroName,BoroCode,the_geom,CTLabel,CT2010,BoroCT2010,CDEligibil,NTACode,NTAName,PUMA,Shape_Leng,Shape_Area,boro_ct2010,Borough,Year,FIPS County Code,DCP Borough Code,Census Tract,Population,pop_density
196,Brooklyn,3,MULTIPOLYGON (((-73.93915724418619 40.62559953...,740.0,74000,3074000,I,BK58,Flatlands,4009,5614.8792,1.747895e+06,3074000,Brooklyn,2010,47,3,074000,3531,2.020145


In [587]:
#building_data["boro_ct2010"] = building_data.census_tract.str.ljust(5, '0')

In [588]:
#building_data["boro_ct2010"] = building_data.boro_ct2010.str.zfill(6)

In [589]:
#del building_data["great"]

In [590]:
#building_data.tail()

In [591]:
#building_data.boro_ct2010.dtype

In [592]:
#building_data["great"] = building_data.boro_ct2010.str.len() > 6

In [593]:
#building_data.shape

In [594]:
#building_data.borough.unique()

In [595]:
#building_data["boro_code"] = building_data.borough.map({"Manhattan": "1", "Bronx": "2", "Brooklyn": "3", \
                                          # "Queens": "4", "Staten Island": "5"})

In [596]:
#building_data["boro_ct2010"] = building_data["boro_code"].astype(str) + building_data["boro_ct2010"].astype(str)

In [597]:
#building_data.tail()

In [598]:
#NYC_shape merge with house data merge with building_data

In [599]:
NYC_shape.shape

(760, 20)

In [600]:
NYC_shape.boro_ct2010.dtype

dtype('int64')

In [601]:
NYC_shape.boro_ct2010 = NYC_shape.boro_ct2010.astype(str)

In [602]:
NYC_shape.boro_ct2010.dtype

dtype('O')

In [603]:
NYC_shape = NYC_shape.merge(house_data, on="boro_ct2010")

In [604]:
NYC_shape.shape

(760, 49)

In [605]:
NYC_shape.head()

,BoroName,BoroCode,the_geom,CTLabel,CT2010,BoroCT2010,CDEligibil,NTACode,NTAName,PUMA,...,TAX_RET,Avg_TransC,REtaxperow,DISP_INC,energy_cos,REtax_ACS,AVG_TTL,LOCALNAME,Shape_Area_y,boro_code
0,Brooklyn,3,MULTIPOLYGON (((-73.99301822153384 40.69746236...,5.01,501,3000501,I,BK09,Brooklyn Heights-Cobble Hill,4004,...,1737.071267,7205.977121,2600.965286,36501.21653,1412.983117,4711,17116.407730,Brooklyn Heights,0.000009,3
1,Brooklyn,3,MULTIPOLYGON (((-74.01266264860631 40.64925555...,20.00,2000,3002000,I,BK32,Sunset Park West,4012,...,574.850160,3064.116727,580.101530,13647.01968,2061.361323,1700,2968.716870,Bush Terminal,0.000016,3
2,Brooklyn,3,MULTIPOLYGON (((-73.98181677448231 40.68732980...,41.00,4100,3004100,I,BK38,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,4004,...,1374.638495,8024.382252,2001.736607,39450.88208,1690.291069,4224,15948.483090,Boerum Hill,0.000017,3
3,Brooklyn,3,MULTIPOLYGON (((-74.03809447949381 40.62058489...,52.02,5202,3005202,I,BK31,Bay Ridge,4013,...,1582.632893,5761.039594,2039.785814,33727.02430,1585.245533,2937,9382.433070,Bay Ridge,0.000015,3
4,Brooklyn,3,MULTIPOLYGON (((-74.02956408409689 40.61983253...,58.00,5800,3005800,E,BK31,Bay Ridge,4013,...,1713.269708,4568.334204,3609.383943,24360.97565,1599.484797,4026,7439.992247,Bay Ridge,0.000021,3


In [606]:
house_data.boro_ct2010 = house_data.boro_ct2010.astype(str)

In [607]:
house_data.boro_ct2010.dtype

dtype('O')

In [608]:
house_data.head()

,FID,STATEFP10,COUNTYFP10,TRACTCE10,GEOID10,INTPTLAT10,INTPTLON10,State,COUNTY,HH_COUNT,...,Avg_TransC,REtaxperow,DISP_INC,energy_cos,REtax_ACS,AVG_TTL,LOCALNAME,Shape_Area,boro_ct2010,boro_code
0,0,34,17,32400,34017032400,40.792844,-74.013482,New Jersey,Hudson County,2187,...,4145.566539,4966.370331,12232.00950,1773.280152,7735,5007.444405,West New York Town,0.000036,nan032400,NaN
1,1,34,17,10100,34017010100,40.691559,-74.110913,New Jersey,Hudson County,2255,...,5969.286255,4490.984109,22077.84024,1876.844806,7917,6146.888195,Bayonne City,0.000106,nan010100,NaN
2,2,34,17,10200,34017010200,40.682103,-74.104573,New Jersey,Hudson County,1218,...,6014.489564,3339.224580,20488.60531,2167.175106,7692,6193.436421,Bayonne City,0.000031,nan010200,NaN
3,3,34,17,10300,34017010300,40.672439,-74.081016,New Jersey,Hudson County,1093,...,4798.787758,3793.386763,15514.61337,2163.964082,7831,4941.564294,Bayonne City,0.000108,nan010300,NaN
4,4,34,17,10400,34017010400,40.670599,-74.089940,New Jersey,Hudson County,1571,...,7611.970058,4478.110461,35131.00534,2114.480847,7918,7838.446154,Bayonne City,0.000234,nan010400,NaN


In [609]:
NYC_shape.head()

,BoroName,BoroCode,the_geom,CTLabel,CT2010,BoroCT2010,CDEligibil,NTACode,NTAName,PUMA,...,TAX_RET,Avg_TransC,REtaxperow,DISP_INC,energy_cos,REtax_ACS,AVG_TTL,LOCALNAME,Shape_Area_y,boro_code
0,Brooklyn,3,MULTIPOLYGON (((-73.99301822153384 40.69746236...,5.01,501,3000501,I,BK09,Brooklyn Heights-Cobble Hill,4004,...,1737.071267,7205.977121,2600.965286,36501.21653,1412.983117,4711,17116.407730,Brooklyn Heights,0.000009,3
1,Brooklyn,3,MULTIPOLYGON (((-74.01266264860631 40.64925555...,20.00,2000,3002000,I,BK32,Sunset Park West,4012,...,574.850160,3064.116727,580.101530,13647.01968,2061.361323,1700,2968.716870,Bush Terminal,0.000016,3
2,Brooklyn,3,MULTIPOLYGON (((-73.98181677448231 40.68732980...,41.00,4100,3004100,I,BK38,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,4004,...,1374.638495,8024.382252,2001.736607,39450.88208,1690.291069,4224,15948.483090,Boerum Hill,0.000017,3
3,Brooklyn,3,MULTIPOLYGON (((-74.03809447949381 40.62058489...,52.02,5202,3005202,I,BK31,Bay Ridge,4013,...,1582.632893,5761.039594,2039.785814,33727.02430,1585.245533,2937,9382.433070,Bay Ridge,0.000015,3
4,Brooklyn,3,MULTIPOLYGON (((-74.02956408409689 40.61983253...,58.00,5800,3005800,E,BK31,Bay Ridge,4013,...,1713.269708,4568.334204,3609.383943,24360.97565,1599.484797,4026,7439.992247,Bay Ridge,0.000021,3


In [610]:
#building_data.boro_ct2010.dtype

In [611]:
#NYC_shape = NYC_shape.merge(building_data, on="boro_ct2010", how="outer")

In [612]:
NYC_shape.shape

(760, 49)

In [613]:
house_data.columns

Index(['FID', 'STATEFP10', 'COUNTYFP10', 'TRACTCE10', 'GEOID10', 'INTPTLAT10',
       'INTPTLON10', 'State', 'COUNTY', 'HH_COUNT', 'OHU_OWN', 'OHU_RENT',
       'PERC_OWN', 'PERC_RENT', 'MHI', 'MED_VAL', 'HOUSINGCOS', 'MCR_ANNUAL',
       'MSMOC_TOT_', 'TAX_RET', 'Avg_TransC', 'REtaxperow', 'DISP_INC',
       'energy_cos', 'REtax_ACS', 'AVG_TTL', 'LOCALNAME', 'Shape_Area',
       'boro_ct2010', 'boro_code'],
      dtype='object')

In [614]:
#building_data.columns

In [623]:
# household_density per 1000 sq feet by census tract
NYC_shape["household_density"] = (NYC_shape.HH_COUNT/NYC_shape.Shape_Area_x) *1000

In [625]:
NYC_shape.head()

,BoroName,BoroCode,the_geom,CTLabel,CT2010,BoroCT2010,CDEligibil,NTACode,NTAName,PUMA,...,Avg_TransC,REtaxperow,DISP_INC,energy_cos,REtax_ACS,AVG_TTL,LOCALNAME,Shape_Area_y,boro_code,household_density
0,Brooklyn,3,MULTIPOLYGON (((-73.99301822153384 40.69746236...,5.01,501,3000501,I,BK09,Brooklyn Heights-Cobble Hill,4004,...,7205.977121,2600.965286,36501.21653,1412.983117,4711,17116.407730,Brooklyn Heights,0.000009,3,2.084288
1,Brooklyn,3,MULTIPOLYGON (((-74.01266264860631 40.64925555...,20.00,2000,3002000,I,BK32,Sunset Park West,4012,...,3064.116727,580.101530,13647.01968,2061.361323,1700,2968.716870,Bush Terminal,0.000016,3,0.269625
2,Brooklyn,3,MULTIPOLYGON (((-73.98181677448231 40.68732980...,41.00,4100,3004100,I,BK38,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,4004,...,8024.382252,2001.736607,39450.88208,1690.291069,4224,15948.483090,Boerum Hill,0.000017,3,0.876099
3,Brooklyn,3,MULTIPOLYGON (((-74.03809447949381 40.62058489...,52.02,5202,3005202,I,BK31,Bay Ridge,4013,...,5761.039594,2039.785814,33727.02430,1585.245533,2937,9382.433070,Bay Ridge,0.000015,3,1.009202
4,Brooklyn,3,MULTIPOLYGON (((-74.02956408409689 40.61983253...,58.00,5800,3005800,E,BK31,Bay Ridge,4013,...,4568.334204,3609.383943,24360.97565,1599.484797,4026,7439.992247,Bay Ridge,0.000021,3,0.755659


In [627]:
#NYC_shape["boro_code"] = NYC_shape.boro_code_x
NYC_shape["Shape_Area"] = NYC_shape.Shape_Area_x

In [628]:
NYC_shape.columns

Index(['BoroName', 'BoroCode', 'the_geom', 'CTLabel', 'CT2010', 'BoroCT2010',
       'CDEligibil', 'NTACode', 'NTAName', 'PUMA', 'Shape_Leng',
       'Shape_Area_x', 'boro_ct2010', 'Borough', 'Year', 'FIPS County Code',
       'DCP Borough Code', 'Census Tract', 'Population', 'pop_density', 'FID',
       'STATEFP10', 'COUNTYFP10', 'TRACTCE10', 'GEOID10', 'INTPTLAT10',
       'INTPTLON10', 'State', 'COUNTY', 'HH_COUNT', 'OHU_OWN', 'OHU_RENT',
       'PERC_OWN', 'PERC_RENT', 'MHI', 'MED_VAL', 'HOUSINGCOS', 'MCR_ANNUAL',
       'MSMOC_TOT_', 'TAX_RET', 'Avg_TransC', 'REtaxperow', 'DISP_INC',
       'energy_cos', 'REtax_ACS', 'AVG_TTL', 'LOCALNAME', 'Shape_Area_y',
       'boro_code', 'household_density', 'Shape_Area'],
      dtype='object')

In [630]:
del NYC_shape["CDEligibil"]
del NYC_shape["Borough"]
del NYC_shape["Census Tract"]
del NYC_shape["DCP Borough Code"]
del NYC_shape["FID"]
del NYC_shape["STATEFP10"]
del NYC_shape["COUNTYFP10"]
del NYC_shape["TRACTCE10"]
del NYC_shape["GEOID10"]
del NYC_shape["Shape_Area"]
del NYC_shape["boro_code"]
del NYC_shape["Shape_Area_x"]
del NYC_shape["Shape_Area_y"]

In [631]:
NYC_shape.head()

,BoroName,BoroCode,the_geom,CTLabel,CT2010,BoroCT2010,NTACode,NTAName,PUMA,Shape_Leng,...,MSMOC_TOT_,TAX_RET,Avg_TransC,REtaxperow,DISP_INC,energy_cos,REtax_ACS,AVG_TTL,LOCALNAME,household_density
0,Brooklyn,3,MULTIPOLYGON (((-73.99301822153384 40.69746236...,5.01,501,3000501,BK09,Brooklyn Heights-Cobble Hill,4004,4828.169912,...,18360,1737.071267,7205.977121,2600.965286,36501.21653,1412.983117,4711,17116.407730,Brooklyn Heights,2.084288
1,Brooklyn,3,MULTIPOLYGON (((-74.01266264860631 40.64925555...,20.00,2000,3002000,BK32,Sunset Park West,4012,5426.067279,...,12852,574.850160,3064.116727,580.101530,13647.01968,2061.361323,1700,2968.716870,Bush Terminal,0.269625
2,Brooklyn,3,MULTIPOLYGON (((-73.98181677448231 40.68732980...,41.00,4100,3004100,BK38,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,4004,5283.856660,...,39360,1374.638495,8024.382252,2001.736607,39450.88208,1690.291069,4224,15948.483090,Boerum Hill,0.876099
3,Brooklyn,3,MULTIPOLYGON (((-74.03809447949381 40.62058489...,52.02,5202,3005202,BK31,Bay Ridge,4013,5190.713818,...,15132,1582.632893,5761.039594,2039.785814,33727.02430,1585.245533,2937,9382.433070,Bay Ridge,1.009202
4,Brooklyn,3,MULTIPOLYGON (((-74.02956408409689 40.61983253...,58.00,5800,3005800,BK31,Bay Ridge,4013,6098.167799,...,15480,1713.269708,4568.334204,3609.383943,24360.97565,1599.484797,4026,7439.992247,Bay Ridge,0.755659


In [632]:
writer = pd.ExcelWriter('MEGA_DATASET.xlsx')
NYC_shape.to_excel(writer)
writer.save()

In [633]:
NYC_shape.Population.describe()

count     760.000000
mean     3295.657895
std      1395.984746
min         0.000000
25%      2298.750000
50%      3219.000000
75%      4136.500000
max      8938.000000
Name: Population, dtype: float64

In [634]:
NYC_shape.pop_density.describe()

count    760.000000
mean       1.738692
std        0.863948
min        0.000000
25%        1.182242
50%        1.689640
75%        2.197788
max        5.625838
Name: pop_density, dtype: float64

In [635]:
NYC_shape.household_density.describe()

count    760.000000
mean       0.634674
std        0.329827
min        0.000000
25%        0.412703
50%        0.611937
75%        0.812231
max        2.356579
Name: household_density, dtype: float64

In [636]:
NYC_shape.MHI.describe()

count       760.000000
mean      48122.368421
std       21700.146358
min           0.000000
25%       34108.000000
50%       43521.000000
75%       59650.000000
max      163147.000000
Name: MHI, dtype: float64

In [418]:
# geenrate points for wifi data (unique bssid by census track for 2016 data) and sjoin with shape maga dataset
# generate regression model
# plot results 